In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local

# Update PATH
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')

# Clone your repo
!git clone https://github.com/chambliss/foodbert.git

# Change directory to the repo
%cd foodbert

# Create the conda environment
!conda env create -f environment.yml

In [ ]:
%%bash
source /usr/local/etc/profile.d/conda.sh && conda activate hf-nlp
pip install -e .
# Now you can run other commands that require the conda environment to be active

Obtaining file:///content/foodbert
  Running setup.py develop for food-extractor


In [ ]:
import numpy as np
from tqdm import tqdm

In [ ]:
from food_extractor.food_model import FoodModel
model = FoodModel("chambliss/distilbert-for-food-extraction")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

In [ ]:
recipes = '/content/drive/MyDrive/datasets/recipes_df_10k_balanced.csv'

In [ ]:
df = pd.read_csv(recipes)

In [ ]:
df.head()

,cooking_method,ingredients,recipe_name,tags,Vegetarian&Desserts,Others&D,Vegetarian,Others,Dairy Free,Gluten Free,Low Carb,Low Fat,Low Sodium
0,['Melt the butter in a saute pan over medium-h...,"['1/4 cup unsalted butter', '2/3 cup brown sug...",Bardstown Distillery Bananas Foster,"Bananas Foster,Banana,Fruit,Bourbon,Dessert,Gl...",1,0,0,1,0,1,0,0,1
1,"['Add broth, sun-dried tomatoes, and butter to...","['1 2/3 cups canned beef broth', '3/4 cup read...",Steak Pinwheels with Sun-Dried Tomato Stuffing...,"Steak,Beef,Meat",0,1,0,1,0,0,0,0,0
2,"['Using a peeler, remove the zest from 1 lemon...","['2 lemons', '4 cups apple cider', 'One 12-oun...",Spiced Bourbon-Ginger Cider,"Bourbon,Fruit,Lemon,Low-Fat,Low Sodium",0,1,0,1,0,0,0,1,1
3,['Bring a large pot of water to a rolling boil...,"['3 tablespoons sesame oil', '2 tablespoons wh...",Asparagus Spears with Sesame,"Vegetarian,Asparagus,Vegetable,Vegan",1,0,1,0,0,0,0,0,0
4,"['Preheat oven to 350 degrees F.', 'In a large...","['3/4 cup unsweetened cocoa, plus more for dus...",Chocolate Stout Cupcakes,"Easy Dessert Recipes,Dessert,Easy,Easy Baking,...",1,0,0,1,0,0,0,0,1


In [ ]:
df = df[df['ingredients'] != '[]']

In [ ]:
df['ingredients'].value_counts()

ingredients
['Banana Pudding', 'Banana Walnut Bread']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          4
['Quick Vanilla Buttercream Frosting', 'Whipped Cream']                                                                                                                                                                                                                

In [ ]:
ingredients_list = df.ingredients

In [ ]:
labels = df[['Vegetarian', 'Vegetarian&Desserts']]

In [ ]:
ner_list = []
for i, recipe in tqdm(enumerate(ingredients_list), total=len(ingredients_list)):
    try:
        ner_list.append([ingredient['text'] for ingredient in model.extract_foods(recipe)[0]['Ingredient']])
    except Exception as e:
        print(i)
        continue

  1%|          | 121/9884 [00:39<27:53,  5.83it/s]

119


  4%|▎         | 364/9884 [01:57<1:03:22,  2.50it/s]

364


  5%|▌         | 527/9884 [02:55<53:14,  2.93it/s]

...

 83%|████████▎ | 8194/9884 [45:02<06:44,  4.17it/s]

8194


 99%|█████████▉| 9783/9884 [53:57<00:27,  3.62it/s]

9783


100%|██████████| 9884/9884 [54:32<00:00,  3.02it/s]


In [ ]:
ner_ingredients = set()
for el in ner_list:
    for ingredient in el:
        ner_ingredients.add(ingredient)

In [ ]:
len(ner_ingredients)

9748

In [ ]:
import csv

with open('/content/drive/MyDrive/ner_ingredients_balanced.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in list(ner_ingredients):
        writer.writerow([item])

In [ ]:
removed_positions = [121, 364, 527, 613, 1054, 2064, 3102, 3367, 5815, 6018, 6174, 6180, 6507, 7921, 8194, 9783]
len(labels)

labels = labels.drop(removed_positions)
df.reset_index(drop=True, inplace=True)

In [ ]:
labels.to_csv('/content/drive/MyDrive/ner_labels_balanced.csv', index=False)

In [ ]:
len(ner_list)

9868

In [ ]:
import csv

with open('/content/drive/MyDrive/ner_recipes_balanced.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in list(ner_list):
        writer.writerow([item])

In [ ]:
len(labels)

9868